# Note Muzicale

### Marculescu Andrei & Stefan Negulescu

Importam librariile de care avem nevoie & definim functia sinus

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd

In [2]:
def sine (amplitude, frequency, time, phase):
    return amplitude * np.sin (2 * np.pi * frequency * time + phase)

Crearea unui semnal cu frecventa 440 si redarea acestuia folosind _sounddevice_

In [3]:
time_of_view = 2     
frequency = 440      
amplitude = 10000
phase = 0
sampling_rate = 44100
sampling_period = 1./sampling_rate  # s
n_samples = time_of_view/sampling_period
time = np.linspace (0, time_of_view, int(n_samples + 1))
tone = sine(amplitude, frequency, time, phase)

In [4]:
sd.default.samplerate = sampling_rate
wav_wave = np.array(tone, dtype=np.int16)
sd.play(wav_wave, blocking=True)
#sd.stop()



# 1. Scrieți tonurile pentru notele muzicale Do, Re, Mi, Fa, Sol, La, Si, Do.


Incepem prin a ne definii un dictionar cu notele muzicale clasice si frecventele acestora, si in plus 2 tipuri de pauze pe care le putem adauga in partitura creata de noi.



In [5]:
musical_notes = {'Do1' :392, 
                 'Re': 440, 
                 'Mi': 494, 
                 'Fa': 523, 
                 'Sol': 587, 
                 'La': 659, 
                 'Si': 740, 
                 'Do2': 784, 
                 'Pause_Small': 10, 
                 'Pause_Big': 15}


Iteram prin dictionar, pentru fiecare element diferit de cele 2 pauze, generam un semnal cu frecventa specifica notei, pe care il adaugam la o lista.

In [6]:
time_of_view = 1
amplitude = 10000
phase = 0
sampling_rate = 44100
sampling_period = 1./sampling_rate 
n_samples = time_of_view/sampling_period
time = np.linspace (0, time_of_view, int(n_samples + 1))

tones = []
for k, v in musical_notes.items():
    if k != 'Pause_Small' and k != 'Pause_Big':
        tones.append(sine(amplitude, v, time, phase))

Pentru fiecare ton din lista, il redam.

In [7]:
sd.default.samplerate = sampling_rate
for i in range(len(tones)):
    wav_wave = np.array(tones[i], dtype=np.int16)
    sd.play(wav_wave, blocking=True)
    
sd.stop()

#  2. Compuneți un cântec simplu clasic (ex. Frère Jacques) într-un singur semnal.

Definim timpi pentru note si cele 2 tipuri de pauze, prin functia _makeSong_ concatenam semnalele generate pentru fiecare nota sau pauza citita din lista _songNotes_ care este data ca parametru functiei

Functia returneaza un singur semnal _song_ pe care il redam.

In [8]:
time_of_view = 0.5
time_of_view_pause_small = 0.15
time_of_view_pause_big = 0.35
phase = 0
amplitude = 10000

sampling_rate = 44100
sampling_period = 1./sampling_rate  # s
n_samples = time_of_view/sampling_period

time = np.linspace (0, time_of_view, int(n_samples + 1))
time_pause_small = np.linspace (0, time_of_view_pause_small, int(n_samples + 1))
time_pause_big = np.linspace (0, time_of_view_pause_big, int(n_samples + 1))

def makeSong(songNotes):
    song = []
    for i in range(len(songNotes)):
        if songNotes[i] == 'Pause_Small':
            song = np.concatenate((song, sine(amplitude, 10, time_pause_small, phase)), axis=None)
        elif songNotes[i] == 'Pause_Big':
            song = np.concatenate((song, sine(amplitude, 15, time_pause_big, phase)), axis=None)
        else:
            song = np.concatenate((song, sine(amplitude, int(musical_notes[songNotes[i]]), time, phase)), axis=None)
    return song
            
songNotes = ['Do1', 'Re', 'Mi', 'Do1', 'Pause_Small', 'Do1', 'Re', 'Mi', 'Do1', 'Pause_Big', 'Mi', 'Fa', 'Sol', 'Pause_Small', 'Mi', 'Fa', 'Sol', 'Pause_Big', 'Sol', 'La', 'Sol', 'Fa', 'Mi', 'Do1', 'Pause_Small', 'Sol', 'La', 'Sol', 'Fa', 'Mi', 'Do1' , 'Pause_Big', 'Do1', 'Sol', 'Do2', 'Pause_Small', 'Do1', 'Sol', 'Do2']
sd.default.samplerate = sampling_rate

wav_wave = np.array(makeSong(songNotes), dtype=np.int16)
sd.play(wav_wave, blocking=True)
sd.stop()

# 3. Citiți o partitură la intrare (folosind LilyPond sau formatul propriu) și produceți semnalul ce conține melodia la ieșire prin compunerea tonurilor asociate notelor automat.

Ne folosim in acest punct de functia _makeSong_ si de dictionarul _musical_notes_ din punctele anterioare. Prin functia _readPartition()_ Citim din fisierul _FrereJacques_ linii, despartim fiecare element de pe linie si obtinem o lista ca _songNotes_ de la punctul 2, partitura returnata este dupa folosita prin functia _makeSong_ pentru a genera semnalul cantecului redat.

Formatul textului este doar din elemente din cheile dictionarului _musical_notes_.

In [9]:
def readPartition():
    partition = []
    with open('FrereJacques.txt') as s:
        for line in s.readlines():
            for note in line.split():
                partition.append(str(note))
    return partition
    
    
    
sd.default.samplerate = sampling_rate
wav_wave = np.array(makeSong(readPartition()), dtype=np.int16)
sd.play(wav_wave, blocking=True)
sd.stop()